In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import glob
import gc
import torch
import json
import random
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gpd
from pathlib import Path
from tqdm.notebook import tqdm
from neuralhydrology.nh_run import start_run, eval_run
from scripts.file_manipulator import (test_rewriter, train_cmip_val,
                                      train_rewriter)
from sklearn.metrics import mean_absolute_error
from scripts.comparison import gauge_cmip

from neuralhydrology.evaluation import get_tester
from neuralhydrology.utils.config import Config

import warnings
warnings.filterwarnings("ignore")

### Read all necessary data

In [8]:
era_input = ['prcp_e5l',  't_max_e5l', 't_min_e5l']
# q_mm_day or lvl_mbs or lvl_sm
hydro_target = 'lvl_sm'
q_h_relation = False

if (hydro_target == 'lvl_mbs') | (hydro_target == 'lvl_sm'):
    static_parameters = ['for_pc_sse', 'crp_pc_sse',
                         'inu_pc_ult', 'ire_pc_sse',
                         'lka_pc_use', 'prm_pc_sse',
                         'pst_pc_sse', 'cly_pc_sav',
                         'slt_pc_sav', 'snd_pc_sav',
                         'kar_pc_sse', 'urb_pc_sse',
                         'gwt_cm_sav', 'lkv_mc_usu',
                         'rev_mc_usu', 'sgr_dk_sav',
                         'slp_dg_sav', 'ws_area',
                         'ele_mt_sav', 'height_bs']
    nc_variable = 'nc_all_h'
    if q_h_relation:
        nc_variable = 'nc_all_q_h'
else:
    static_parameters = ['for_pc_sse', 'crp_pc_sse',
                         'inu_pc_ult', 'ire_pc_sse',
                         'lka_pc_use', 'prm_pc_sse',
                         'pst_pc_sse', 'cly_pc_sav',
                         'slt_pc_sav', 'snd_pc_sav',
                         'kar_pc_sse', 'urb_pc_sse',
                         'gwt_cm_sav', 'lkv_mc_usu',
                         'rev_mc_usu', 'sgr_dk_sav',
                         'slp_dg_sav', 'ws_area',
                         'ele_mt_sav']
    nc_variable = 'nc_all_q'

ws_file = gpd.read_file('../geo_data/great_db/geometry/russia_ws.gpkg')
ws_file = ws_file.set_index('gauge_id')

# define dates for data split
train_start_date = '01/01/2009'
train_end_date = '31/12/2016'
validation_start_date = '01/01/2017'
validation_end_date = '31/12/2018'
test_start_date = '01/01/2019'
test_end_date = '31/12/2020'
# dates for cmip comparisson
compare_start = '01/01/2015'
compare_end = '31/12/2020'


### compare CMIP models and select best ones

In [4]:
compare_list = dict()
for f in tqdm(glob.glob(f'../geo_data/great_db/{nc_variable}/*.nc')):
    gauge_id = f.split('/')[-1][:-3]
    compare_element, model_order = gauge_cmip(
        gauge_id=gauge_id,
        cmip_storage='../geo_data/cmip_concat_21_09/cmip_245/',
        era_storage=f'../geo_data/great_db/{nc_variable}/',
        compare_start=compare_start,
        compare_end=compare_end)
    compare_list[gauge_id] = compare_element

number_df = pd.concat([i['number'].reset_index(drop=True)
                       for gauge, i in compare_list.items()],
                      axis=1)
error_df = pd.concat([i['error'].reset_index(drop=True)
                      for gauge, i in compare_list.items()],
                     axis=1)
res_models = np.unique(number_df.mode(axis=1).iloc[:7, 0].values)

model_dict = dict()
for i, order in enumerate(res_models):
    model_dict[i+1] = model_order[order]
    
with open('cmip_ranks.json', 'w') as fp:
    json.dump(model_dict, fp)

A Jupyter Widget

### Read rankings to prepare input data

In [4]:
with open('./cmip_ranks.json') as json_file:
    cmip_models = json.load(json_file)
cmip_models = list(cmip_models.values())

meteo_inputs = dict()
for model in cmip_models:
    meteo_inputs[model] = [f'precipitation_{model}',
                           f'daily_maximum_near_surface_air_temperature_{model}',
                           f'daily_minimum_near_surface_air_temperature_{model}']
# time series directory
ts_dir = Path('../geo_data/time_series')

### NeuralHydrology train data
Relies on config file. **Main** attention should be payed to **data_dir** argument
This folder is the reference to place where all required data are stored

<br><br/>
Framework automaticcally seek for **time_series** folder which should contain data **for each gauge** in desired experiment
and has all available meteo data in **.nc** format

To inlude static parameters **data_dir** should also has **attributes** folder with **.csv** file for each used gauge as an index of this file

CMIP data of desired model of **cmip_cols** parameter will replace **validation** data of **era_cols**

#### With CMIP

In [5]:
# write files for train procedure
print(f'train data for {hydro_target} with {nc_variable} initial data')
train_cmip_val(era_pathes=glob.glob(
    f'../geo_data/great_db/{nc_variable}/*.nc'),
               ts_dir=ts_dir,
               hydro_target=hydro_target,
               area_index=ws_file.index,
               era_cols=era_input,
               cmip_cols=meteo_inputs['noresm2_mm'],
               cmip_storage='../geo_data/cmip_concat_21_09/cmip_245/',
               val_start=validation_start_date,
               val_end=test_end_date)

train data for lvl_sm with nc_all_h initial data


#### Just ERA

In [10]:
# write files for train procedure
print(f'train data for {hydro_target} with {nc_variable} initial data')
train_rewriter(era_pathes=glob.glob(
    f'../geo_data/great_db/{nc_variable}/*.nc'),
    ts_dir=ts_dir,
    hydro_target=hydro_target,
    area_index=ws_file.index,
    predictors=era_input)

train data for lvl_sm with nc_all_h initial data


### Config setup

Here we define files for **train**, **validation** and **test**

It's possible to use different gauges for certain steps of model learn procedure,

but in this example we'll use every possible gauge and split it **not in space**, but with **different time periods**

'**train_start_date**': '01/01/2009',

'**train_end_date**': '31/12/2016',

'**validation_start_date**': '01/01/2017',

'**validation_end_date**': '31/12/2018',

'**test_start_date**': '01/01/2019',

'**test_end_date**': '31/12/2025'


In [11]:
# define variables require to perform hindcast
gauges = [file.split('/')[-1][:-3] for
          file in glob.glob(f'{ts_dir}/*.nc')]
random.shuffle(gauges)
gauge_size = len(gauges)

# with open('./every_basin.txt', 'w') as the_file:
#     for gauge_name in gauges:
#         the_file.write(f'{gauge_name}\n')

train_gauges = gauges[
    :int(len(gauges) * 0.8)]
with open('./basins_train.txt', 'w') as the_file:
    for gauge_name in train_gauges:
        the_file.write(f'{gauge_name}\n')

val_gauges = gauges[
    int(gauge_size * 0.8):int(gauge_size * 0.8) + int(gauge_size * 0.1)]
with open('./basins_val.txt', 'w') as the_file:
    for gauge_name in val_gauges:
        the_file.write(f'{gauge_name}\n')

test_gauges = gauges[
    int(gauge_size * 0.8) + int(gauge_size * 0.1):]
with open('./basins_test.txt', 'w') as the_file:
    for gauge_name in test_gauges:
        the_file.write(f'{gauge_name}\n')

cfg = Config(Path('./model_config.yml'))
# base model type [cudalstm, customlstm, ealstm, embcudalstm, mtslstm, gru, transformer]
# (has to match the if statement in modelzoo/__init__.py)
model_name = 'cudalstm'
comment = f'era5_land'
cfg.update_config(yml_path_or_dict={
    # define storage and experiment
    'experiment_name': f'{model_name}_{hydro_target}_{comment}',
    'model': f'{model_name}',
    'run_dir': './model_runs/',
    'data_dir': '../geo_data/',
    # define inner parameters
    'static_attributes': static_parameters,
    'dynamic_inputs': era_input,
    # 'hindcast_inputs': era_input,
    # 'forecast_inputs': era_input,
    'target_variables': [hydro_target],
    # 'dynamics_embedding': {'type': 'fc', 'hiddens': [128, 64, 256],
    #                       'activation': 'tanh', 'dropout': 0.2},
    # 'statics_embedding': {'type': 'fc', 'hiddens': [128, 64, 256],
    #                      'activation': 'tanh', 'dropout': 0.2},    
    # define files with gauge data
    'train_basin_file': './every_basin.txt',
    'validate_n_random_basins': gauge_size,
    'validation_basin_file': './every_basin.txt',
    'test_basin_file': './every_basin.txt',
    # define time periods
    # 'seq_length': 14,
    # 'forecast_seq_length': 10,
    'train_start_date': train_start_date,
    'train_end_date': train_end_date,
    'validation_start_date': validation_start_date,
    'validation_end_date': validation_end_date,
    'test_start_date': test_start_date,
    'test_end_date': test_end_date})
cfg.dump_config(folder=Path('./launch_configs'),
                filename=f'{model_name}_{hydro_target}_{comment}.yml')

In [12]:
if torch.cuda.is_available():
    start_run(config_file=Path(
        f'./launch_configs/{model_name}_{hydro_target}_{comment}.yml'))

2023-09-25 04:46:47,508: Logging to model_runs/cudalstm_lvl_sm_era5_land_2509_044647/output.log initialized.
2023-09-25 04:46:47,508: ### Folder structure created at model_runs/cudalstm_lvl_sm_era5_land_2509_044647
2023-09-25 04:46:47,509: ### Run configurations for cudalstm_lvl_sm_era5_land
2023-09-25 04:46:47,509: additional_feature_files: None
2023-09-25 04:46:47,509: batch_size: 256
2023-09-25 04:46:47,510: cache_validation_data: True
2023-09-25 04:46:47,510: checkpoint_path: None
2023-09-25 04:46:47,511: clip_gradient_norm: 1
2023-09-25 04:46:47,511: data_dir: ../geo_data
2023-09-25 04:46:47,512: dataset: generic
2023-09-25 04:46:47,512: device: cuda:0
2023-09-25 04:46:47,512: dynamic_inputs: ['prcp_e5l', 't_max_e5l', 't_min_e5l']
2023-09-25 04:46:47,513: dynamics_embedding: None
2023-09-25 04:46:47,513: epochs: 30
2023-09-25 04:46:47,513: evolving_attributes: None
2023-09-25 04:46:47,514: experiment_name: cudalstm_lvl_sm_era5_land
2023-09-25 04:46:47,514: forecast_hidden_size: 25

2023-09-25 04:46:47,771: Loading basin data into xarray data set.
100%|██████████| 1404/1404 [00:22<00:00, 61.30it/s]
2023-09-25 04:47:12,251: Create lookup table and convert to pytorch tensor
# Epoch 1: 100%|██████████| 16026/16026 [08:58<00:00, 29.74it/s, Loss: 0.2170]
2023-09-25 04:56:27,750: Epoch 1 average loss: avg_loss: 0.40957, avg_total_loss: 0.40957
# Validation: 100%|██████████| 1404/1404 [01:46<00:00, 13.20it/s]
2023-09-25 04:58:14,100: Stored metrics at model_runs/cudalstm_lvl_sm_era5_land_2509_044647/validation/model_epoch001/validation_metrics.csv
2023-09-25 04:58:14,259: Stored results at model_runs/cudalstm_lvl_sm_era5_land_2509_044647/validation/model_epoch001/validation_results.p
2023-09-25 04:58:14,273: Epoch 1 average validation loss: 0.21690 -- Median validation metrics: avg_loss: 0.21690, NSE: 0.33375, RMSE: 40.20328
# Epoch 2: 100%|██████████| 16026/16026 [08:59<00:00, 29.72it/s, Loss: 0.1609]
2023-09-25 05:07:13,518: Epoch 2 average loss: avg_loss: 0.22581, avg